In the Kaggle notebook containing the Olist Brazillian E-commerce sales, the author has provided a schema which shows the relationships of all datasets. However, in this project, I assumed that no schema is provided and I have to start from scratch. I used python for the inspection of the data and to do initial cleaning before uploading the datasets in MySQL. 

In [28]:
import pandas as pd
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [39]:
#loading datasets of Olist sales
datasets = {
    "customers": pd.read_csv("olist_customers_dataset.csv"),
    "geolocation": pd.read_csv("olist_geolocation_dataset.csv"),
    "order_info": pd.read_csv("olist_orders_dataset.csv"),
    "order_items": pd.read_csv("olist_order_items_dataset.csv"),
    "order_payments": pd.read_csv("olist_order_payments_dataset.csv"),
    "order_reviews": pd.read_csv("olist_order_reviews_dataset.csv"),
    "products": pd.read_csv("olist_products_dataset.csv"),
    "sellers": pd.read_csv("olist_sellers_dataset.csv"),
    "product_category_translation": pd.read_csv(
        "product_category_name_translation.csv"
    )
}


To start, we need to inspect the content of the csv tables:
-identify column names,number of columns, number of rows
-identify data types and format
-identify null values, missing values, duplicates
-identify primary keys and foreign keys

Using these information, we can now draft the schema of our database.

In [40]:
#Loop function collecting the datasets information
for name, df in datasets.items():
    print(f"\n{'='*60}")
    print(f"DATASET: {name.upper()}")
    print(f"{'='*60}")
    
    #create a dataframe to indetify data types, null content, row count, and unique values
    summary_df = pd.DataFrame({
    "dtype": df.dtypes,
    "row_count": len(df),
    "null_count": df.isna().sum(), #identify missing values or na values to be removed
    "unique_count": df.nunique(),  #helps in identifying primary and feature keys, d
    "percent_unique": (df.nunique() / len(df)) * 100,
}).sort_values("unique_count", ascending=False)

    display(summary_df)
    
   #Counts duplicate rows to be removed 
    print("Duplicate rows:", df.duplicated().sum())

    print("\nShape (rows, columns):", df.shape)    
    # Prints first 5 rows, inspect the data formatting
    display(df.head())


DATASET: CUSTOMERS


,dtype,row_count,null_count,unique_count,percent_unique
customer_id,object,99441,0,99441,100.000000
customer_unique_id,object,99441,0,96096,96.636196
customer_zip_code_prefix,int64,99441,0,14994,15.078288
customer_city,object,99441,0,4119,4.142155
customer_state,object,99441,0,27,0.027152


Duplicate rows: 0

Shape (rows, columns): (99441, 5)


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP



DATASET: GEOLOCATION


...

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP



DATASET: PRODUCT_CATEGORY_TRANSLATION


,dtype,row_count,null_count,unique_count,percent_unique
product_category_name,object,71,0,71,100.0
product_category_name_english,object,71,0,71,100.0


Duplicate rows: 0

Shape (rows, columns): (71, 2)


,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


By analyzing the data information in all tables, we now have the idea on what information can we infer from the datasets as well as the schema of our database. 

Customers Dataset
For the customers dataset, most columns are retained as they provide essential customer and location information. However, the customer_zip_code_prefix column is removed since city and state fields are sufficient for location-based analysis at the business level.
Additionally, a noticeable difference between customer_id and customer_unique_id (approximately 3,000+ records) suggests that multiple purchases were made by the same customers over time. This distinction is important for understanding repeat customer behavior versus individual transactions.

Geolocation Dataset
The geolocation dataset serves as a link between customers and sellers through the zip code prefix. However, given the scope of this analysis, precise geographic coordinates are not required. City- and state-level information from the customers and sellers datasets provides sufficient granularity for business insights. As such, the geolocation dataset is excluded from further analysis to reduce redundancy and complexity.

Orders-Related Datasets
The datasets orders, order_items, order_payments, and order_reviews are all connected through the common key order_id.

    The orders dataset contains information related to order status, purchase timestamps, delivery timelines, and fulfillment outcomes.

    The order_items dataset captures the number of items per order, shipping costs, and links orders to both products and sellers.

    The order_payments dataset provides details on payment methods, installment counts, and transaction values.

    The order_reviews dataset contains customer feedback related to completed orders.

Within the order_reviews dataset, columns such as review answer timestamps are review titles are dropped. These fields are not directly required for the current business analysis. I also dropped the message since it was not available in English. However, if it were, python can be used to extract keywords to identify common issues associated with low review scores.

Products Dataset
In the products dataset, product category names are originally provided in Portuguese. I can use the product_category dataset to translate these category names to English. 
I removed the product name length as it wont contribute to significant business insight. However, it can be interesting to explore potential correlations of product description length and product dimensions with order success. For simplicity, product length, height, and width can be combined into a single product dimension metric.
The remaining attributes—such as product dimensions and weight—are retained to explore potential correlations with order success, shipping costs, and customer satisfaction. For simplicity, product length, height, and width are combined into a single product dimension metric.

Sellers Dataset
For the sellers dataset, all columns except the zipcode are retained. Each field provides relevant information for analyzing seller distribution and performance.

In [41]:
# =========================
# Update Customers dataset
# =========================
customers = datasets["customers"]
customers = customers.drop(columns=["customer_zip_code_prefix"])
datasets["customers"] = customers
display(customers.head(5))

# =========================
# Update Order Reviews dataset
# =========================
reviews = datasets["order_reviews"]
reviews = reviews.drop(columns=[
    "review_comment_title",
    "review_comment_message",
    "review_creation_date",
    "review_answer_timestamp"
])
datasets["order_reviews"] = reviews
display(reviews.head(5))

# =========================
# Update Products dataset
# =========================
products = datasets["products"]
category_translation = datasets["product_category_translation"]

# Create translation dictionary
category_map = dict(
    zip(
        category_translation["product_category_name"],
        category_translation["product_category_name_english"]
    )
)

# Translate product category names to English
products["product_category_name"] = (
    products["product_category_name"]
    .map(category_map)
    .fillna(products["product_category_name"])
)

# Create product dimension (volume)
products["product_dimension_cm3"] = (
    products["product_height_cm"]
    * products["product_length_cm"]
    * products["product_width_cm"]
)

# Drop original dimension columns
products = products.drop(columns=[
    "product_height_cm",
    "product_length_cm",
    "product_width_cm"
])

datasets["products"] = products
display(products.head(5))


# =========================
# Update Sellers dataset
# =========================
sellers = datasets["sellers"]
sellers = sellers.drop(columns=["seller_zip_code_prefix"])
datasets["sellers"] = sellers

print("Updated SELLERS dataset:")
display(sellers.head(5))


,customer_id,customer_unique_id,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,campinas,SP


,review_id,order_id,review_score
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_dimension_cm3
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,40.0,287.0,1.0,225.0,2240.0
1,3aa071139cb16b67ca9e5dea641aaa2f,art,44.0,276.0,1.0,1000.0,10800.0
2,96bd76ec8810374ed1b65e291975717f,sports_leisure,46.0,250.0,1.0,154.0,2430.0
3,cef67bcfe19066a932b7673e239eb23d,baby,27.0,261.0,1.0,371.0,2704.0
4,9dc1a7de274444849c219cff195d0b71,housewares,37.0,402.0,4.0,625.0,4420.0


Updated SELLERS dataset:


,seller_id,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,braganca paulista,SP


In [ ]:
#export new csv files for updated datasets

# Folder for output files
output_folder = "updated_olist_datasets"
os.makedirs(output_folder, exist_ok=True)

# list of updated datasets
datasets_to_export = [
    "customers",
    "order_reviews",
    "products",
    "sellers"
]

for name in datasets_to_export:
    df = datasets[name]
    output_path = os.path.join(output_folder, f"{name}_updated.csv")
    df.to_csv(output_path, index=False)
    print(f"Saved {name} to {output_path}")


The next step is to parse the timestamps. While skimming through the data, the timestamps seemed to be in onne format (yyyy:mm:dd HH:MM:SS), it is safer to perform this step as it is harder to clean timestamps in MySQL.

In [ ]:
order_info = datasets["order_info"]
order_info_dates = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]
order_items= datasets["order_items"]
order_items_dates = [
    "shipping_limit_date"
]

#Force all columns in datetime format
for col in order_info_dates:
    order_info[col] = pd.to_datetime(order_info[col], errors="coerce")

for col in order_items_dates:
    order_items[col] = pd.to_datetime(order_items[col], errors="coerce")

#Check for invalid timestamps

print("Invalid timestamps in order_info:")
for col in order_info_dates:
    invalid_count = order_info[col].isna().sum()
    print(f"{col}: {invalid_count} invalid rows")

print("\nInvalid timestamps in order_items:")
for col in order_items_dates:
    invalid_count = order_items[col].isna().sum()
    print(f"{col}: {invalid_count} invalid rows")

As seen, the number of rows with invalid timestamps match the number of null values in the dataset info for order_info last time. Hence, all of the timestamps are already in correct format. The datasets are ready for importing to mysql.

In [ ]:
#shows distribution of avg budget and frquency of purchases of customers
orders= datasets["order_info"]
payments = datasets["order_payments"]

# Create dataframe merging customers, orders, and payments
df = pd.merge(customers, orders, on='customer_id')
df = pd.merge(df, payments, on='order_id') #ensures that only paid orders are included
df = df[df['order_status'].isin(['delivered', 'shipped', 'invoiced'])] #filter for successful orders

# Calculate metrics per unique customer
customer_data = df.groupby('customer_unique_id').agg({
    'order_id': 'nunique',     # Frequency
    'payment_value': 'mean'    # Average Budget
}).rename(columns={'order_id': 'frequency', 'payment_value': 'avg_budget'})


# Plotting Distribution
plt.figure(figsize=(15, 6))

# Plot 1: Frequency Distribution 
plt.subplot(1, 2, 1)
sns.countplot(data=customer_data, x='frequency', palette='viridis')
plt.title('Distribution of Order Frequency')
plt.xlabel('Number of Orders')
plt.ylabel('Number of Customers')
plt.yscale('log') # Log scale because frequency = 1 is usually massive

# Plot 2: Budget Distribution 
plt.subplot(1, 2, 2)
# I limit the X-axis to the 95th percentile to ignore extreme outliers for a better view
limit = customer_data['avg_budget'].quantile(0.95)
sns.histplot(customer_data[customer_data['avg_budget'] <= limit]['avg_budget'], bins=50, kde=True, color='teal')
plt.title('Distribution of Average Budget')
plt.xlabel('Average Payment Value')
plt.ylabel('Number of Customers')

plt.tight_layout()
plt.show()